In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Administrator\\Demand Forecasting 24122024\\End-to-End-Demand-Forecasting-for-Wooden-Pallets\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Administrator\\Demand Forecasting 24122024\\End-to-End-Demand-Forecasting-for-Wooden-Pallets'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from DemandForecasting_WoodenPallets.constant import *
from DemandForecasting_WoodenPallets.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from DemandForecasting_WoodenPallets import logger
from DemandForecasting_WoodenPallets.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config



    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with the following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-14 10:00:49,893: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-14 10:00:49,895: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-14 10:00:49,898: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-14 10:00:49,899: INFO: common: created directory at: artifacts]
[2025-03-14 10:00:49,902: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-14 10:00:52,575: INFO: 4019116171: artifacts/data_ingestion/data.zip download! with the following info: 
Connection: close
Content-Length: 205917
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "947685d111a06f5bdb7cc1495fd3962b07e2ad1189be13f42e53838699513732"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: BE12:F0F30:3FCF17:8D2EAD:67D38DD2
Accept-Ranges: bytes
Date: Fri